In [1]:
!pip install datasets
!pip install transformers
!sudo apt-get install swig
!sudo pip install jamspell
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state info

In [2]:
import string
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import jamspell
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import torch
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, set_seed

from sklearn.metrics import classification_report, f1_score
from scipy.special import softmax
from tqdm import tqdm

import datasets
from datasets import load_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
dataset = datasets.load_dataset('tweet_eval', 'sentiment')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')
lemmatizer= WordNetLemmatizer()
punct = ''.join([i for i in string.punctuation if i not in [',','?','!','.']])

def light_clean(x):
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos las palabras que empiezan por @.
  x = re.sub(r'@[\w]+', '', x)
  # Signos de puntuacion menos. 
  x = x.translate(str.maketrans('', '', punct))
  x = x.replace('#', '')
  # Spell check
  x = corrector.FixFragment(x)
  return x

In [5]:
# vectorizamos la funcion utilizada para limpiar el texto.
clean1_vect = np.vectorize(light_clean)

In [6]:
# Aplicamos la funcion en el training, validation y test set.
data = dataset.map(lambda e: {"text_clean": clean1_vect(e['text'])}, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [7]:
# Modelo a utilizar
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [8]:
# Iniciamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True, max_length=512)
#config = AutoConfig.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
# Inicializamos nuestro modelo.
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
#tokenizamos 
data = data.map(lambda e: tokenizer(e['text'], truncation=True, max_length=512), batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [11]:
# Sets en Formato Dict, datos listos para darselos al modelo.
train_dataset = data['train']
val_dataset = data['validation']
test_dataset = data['test']

In [12]:
# Class distribution -> Unbalanced data.
train_df = pd.DataFrame(dataset['train'])
train_df['label'].value_counts(normalize=True).sort_index()

0    0.155497
1    0.453206
2    0.391297
Name: label, dtype: float64

In [13]:
class_weights = (1 - (train_df['label'].value_counts().sort_index() / len(train_df))).values
class_weights = torch.from_numpy(class_weights).float().to("cuda")

In [14]:
train_dataset = train_dataset.rename_column('label', 'labels')

In [15]:
class WeightLossTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    # feed inputs to the model and extract logits
    outputs = model(**inputs)
    logits = outputs.get('logits')
    # extract label
    labels = inputs.get('labels')
    # define loss function with lass weights
    loss_func = nn.CrossEntropyLoss(weight=class_weights)
    # compute loss
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

In [16]:
modelFineTuned = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
batch_size = 64
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='./1',
    num_train_epochs=5, 
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    warmup_steps = 50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True,
    fp16= True,
)

In [18]:
trainer1 = Trainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer)

Using cuda_amp half precision backend


In [19]:
trainer1.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3565
  Number of trainable parameters = 124647939
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` m

Epoch,Training Loss,Validation Loss
1,0.538100,0.532114
2,0.474800,0.541974
3,0.420800,0.557826
4,0.375400,0.597373
5,0.345500,0.617086


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to ./1/checkpoint-713
Configuration saved in ./1/checkpoint-713/config.json
Model weights saved in ./1/checkpoint-713/pytorch_model.bin
tokenizer config file saved in ./1/checkpoint-713/tokenizer_config.json
Special tokens file saved in ./1/checkpoint-713/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2

TrainOutput(global_step=3565, training_loss=0.4307915033832673, metrics={'train_runtime': 307.7399, 'train_samples_per_second': 741.129, 'train_steps_per_second': 11.584, 'total_flos': 5847583166351514.0, 'train_loss': 0.4307915033832673, 'epoch': 5.0})

In [20]:
test_preds_raw, test_labels , _ = trainer1.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12284
  Batch size = 64


              precision    recall  f1-score   support

           0      0.763     0.682     0.720      3972
           1      0.718     0.725     0.722      5937
           2      0.665     0.768     0.713      2375

    accuracy                          0.719     12284
   macro avg      0.716     0.725     0.718     12284
weighted avg      0.723     0.719     0.719     12284



In [21]:
batch_size = 32
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='./2',
    num_train_epochs=5, 
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    warmup_steps = 50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True,
    fp16= True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
trainer2 = Trainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer)

Using cuda_amp half precision backend


In [23]:
trainer2.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7130
  Number of trainable parameters = 124647939


Epoch,Training Loss,Validation Loss
1,0.489400,0.538338
2,0.408700,0.592072
3,0.337600,0.657339
4,0.281000,0.715178
5,0.235900,0.774023


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to ./2/checkpoint-1426
Configuration saved in ./2/checkpoint-1426/config.json
Model weights saved in ./2/checkpoint-1426/pytorch_model.bin
tokenizer config file saved in ./2/checkpoint-1426/tokenizer_config.json
Special tokens file saved in ./2/checkpoint-1426/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num example

TrainOutput(global_step=7130, training_loss=0.3504653216077101, metrics={'train_runtime': 557.1457, 'train_samples_per_second': 409.363, 'train_steps_per_second': 12.797, 'total_flos': 5475120942781224.0, 'train_loss': 0.3504653216077101, 'epoch': 5.0})

In [24]:
test_preds_raw, test_labels , _ = trainer2.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12284
  Batch size = 32


              precision    recall  f1-score   support

           0      0.744     0.698     0.720      3972
           1      0.717     0.716     0.716      5937
           2      0.675     0.747     0.709      2375

    accuracy                          0.716     12284
   macro avg      0.712     0.720     0.715     12284
weighted avg      0.718     0.716     0.716     12284



In [25]:
batch_size = 4
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='./3',
    num_train_epochs=5, 
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    warmup_steps = 50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True,
    fp16= True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [26]:
trainer3 = Trainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer)

Using cuda_amp half precision backend


In [28]:
trainer3.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 57020
  Number of trainable parameters = 124647939


Epoch,Training Loss,Validation Loss
1,0.615300,0.893393
2,0.547100,1.015128
3,0.413500,1.335874
4,0.290200,1.566877
5,0.207800,1.725696


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 4
Saving model checkpoint to ./3/checkpoint-11404
Configuration saved in ./3/checkpoint-11404/config.json
Model weights saved in ./3/checkpoint-11404/pytorch_model.bin
tokenizer config file saved in ./3/checkpoint-11404/tokenizer_config.json
Special tokens file saved in ./3/checkpoint-11404/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num exa

TrainOutput(global_step=57020, training_loss=0.41477142014197405, metrics={'train_runtime': 4142.0939, 'train_samples_per_second': 55.063, 'train_steps_per_second': 13.766, 'total_flos': 4361915308723938.0, 'train_loss': 0.41477142014197405, 'epoch': 5.0})

In [29]:
test_preds_raw, test_labels , _ = trainer3.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12284
  Batch size = 4


              precision    recall  f1-score   support

           0      0.687     0.798     0.739      3972
           1      0.762     0.610     0.677      5937
           2      0.640     0.787     0.706      2375

    accuracy                          0.705     12284
   macro avg      0.696     0.732     0.707     12284
weighted avg      0.714     0.705     0.703     12284



In [33]:
batch_size = 64
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='./4',
    num_train_epochs=5, 
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    warmup_steps = 50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True,
    fp16= True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
trainer4 = Trainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer)

Using cuda_amp half precision backend


In [35]:
trainer4.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3565
  Number of trainable parameters = 124647939


Epoch,Training Loss,Validation Loss
1,0.129800,1.026670
2,0.121800,0.827610
3,0.122200,1.086607
4,0.070100,1.449197
5,0.040900,1.579758


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to ./4/checkpoint-713
Configuration saved in ./4/checkpoint-713/config.json
Model weights saved in ./4/checkpoint-713/pytorch_model.bin
tokenizer config file saved in ./4/checkpoint-713/tokenizer_config.json
Special tokens file saved in ./4/checkpoint-713/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2

TrainOutput(global_step=3565, training_loss=0.09693868158575863, metrics={'train_runtime': 307.6641, 'train_samples_per_second': 741.312, 'train_steps_per_second': 11.587, 'total_flos': 5847583166351514.0, 'train_loss': 0.09693868158575863, 'epoch': 5.0})

In [36]:
test_preds_raw, test_labels , _ = trainer4.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12284
  Batch size = 64


              precision    recall  f1-score   support

           0      0.746     0.650     0.694      3972
           1      0.690     0.697     0.693      5937
           2      0.628     0.748     0.683      2375

    accuracy                          0.691     12284
   macro avg      0.688     0.698     0.690     12284
weighted avg      0.696     0.691     0.692     12284



In [37]:
batch_size = 64
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='./5',
    num_train_epochs=5, 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    warmup_steps = 50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True,
    fp16= True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
trainer5 = Trainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer)

Using cuda_amp half precision backend


In [39]:
trainer5.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3565
  Number of trainable parameters = 124647939


Epoch,Training Loss,Validation Loss
1,0.036600,1.565906
2,0.027500,1.628891
3,0.083600,1.246394
4,0.054400,1.438861
5,0.034800,1.616294


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to ./5/checkpoint-713
Configuration saved in ./5/checkpoint-713/config.json
Model weights saved in ./5/checkpoint-713/pytorch_model.bin
tokenizer config file saved in ./5/checkpoint-713/tokenizer_config.json
Special tokens file saved in ./5/checkpoint-713/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2

TrainOutput(global_step=3565, training_loss=0.04740923372257976, metrics={'train_runtime': 308.2991, 'train_samples_per_second': 739.785, 'train_steps_per_second': 11.563, 'total_flos': 5847583166351514.0, 'train_loss': 0.04740923372257976, 'epoch': 5.0})

In [40]:
test_preds_raw, test_labels , _ = trainer5.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, text_clean. If text, text_clean are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12284
  Batch size = 64


              precision    recall  f1-score   support

           0      0.687     0.764     0.724      3972
           1      0.722     0.648     0.683      5937
           2      0.668     0.715     0.691      2375

    accuracy                          0.698     12284
   macro avg      0.693     0.709     0.699     12284
weighted avg      0.700     0.698     0.698     12284

